In [2]:
#One worker per core 
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *

sc=SparkContext(master="local[2]")
spark = SparkSession(sc)

#from pyspark.sql.session import SparkSession
#sc = SparkContext.getOrCreate()
#spark = SparkSession(sc)
#sc.stop()
import pandas as pd 
import numpy as np 

In [13]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

In [4]:
csv=spark.read.csv("data/flights.csv", header=True, inferSchema=True)

In [5]:
data = csv.select("DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay", ((col("ArrDelay") > 15).cast("Int").alias("Late")))
data.show(3)

+----------+---------+---------------+-------------+--------+----+
|DayofMonth|DayOfWeek|OriginAirportID|DestAirportID|DepDelay|Late|
+----------+---------+---------------+-------------+--------+----+
|        19|        5|          11433|        13303|      -3|   0|
|        19|        5|          14869|        12478|       0|   0|
|        19|        5|          14057|        14869|      -4|   0|
+----------+---------+---------------+-------------+--------+----+
only showing top 3 rows



In [6]:
splits = data.randomSplit([0.7, 0.3])
train=splits[0]
test=splits[1]
print("Train: ",train.count(),"Test: ",test.count())

Train:  1891140 Test:  811078


In [7]:
#Transforming training set 
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
training = assembler.transform(train).select(col("features"), col("Late").alias("label"))
training.show(5)
#type(assembler)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    0|
|[1.0,1.0,10140.0,...|    1|
|[1.0,1.0,10140.0,...|    0|
+--------------------+-----+
only showing top 5 rows



In [8]:
#Transform test set 
testing = assembler.transform(test).select(col("features"), col("Late").alias("trueLabel"))
testing.show(5)

+--------------------+---------+
|            features|trueLabel|
+--------------------+---------+
|[1.0,1.0,10140.0,...|        0|
|[1.0,1.0,10140.0,...|        0|
|[1.0,1.0,10140.0,...|        0|
|[1.0,1.0,10140.0,...|        0|
|[1.0,1.0,10140.0,...|        0|
+--------------------+---------+
only showing top 5 rows



In [9]:
#Model fitting for trainning 
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
model = lr.fit(training)
print ("Model trained!")

Model trained!


In [10]:
#Test the model 
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "probability", "trueLabel")
predicted.show(5)

+--------------------+----------+--------------------+---------+
|            features|prediction|         probability|trueLabel|
+--------------------+----------+--------------------+---------+
|[1.0,1.0,10140.0,...|       0.0|[0.83148650576687...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.82751804230989...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.81538104370183...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.80678280971491...|        0|
|[1.0,1.0,10140.0,...|       0.0|[0.83383973947051...|        0|
+--------------------+----------+--------------------+---------+
only showing top 5 rows



In [11]:
predicted.filter("prediction=1.0").select("prediction","trueLabel","probability").show(10)

+----------+---------+--------------------+
|prediction|trueLabel|         probability|
+----------+---------+--------------------+
|       1.0|        1|[0.48474770403796...|
|       1.0|        1|[0.46772075549981...|
|       1.0|        1|[0.48540791245505...|
|       1.0|        1|[0.48640536203948...|
|       1.0|        1|[0.27497794217185...|
|       1.0|        1|[0.41823730629133...|
|       1.0|        1|[0.41194462357452...|
|       1.0|        1|[0.48468344063720...|
|       1.0|        1|[0.28268631971707...|
|       1.0|        1|[0.00239939081247...|
+----------+---------+--------------------+
only showing top 10 rows



In [17]:
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
pipeline = Pipeline(stages=[assembler, lr])
model = pipeline.fit(train)

IllegalArgumentException: 'Field "label" does not exist.'